# MRI maps

In [1]:
%cd work/notebooks

/home/jovyan/work/notebooks


In [2]:
# Import modules and R
import numpy as np, pandas as pd
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

In [69]:
import sys
sys.path.append('/home/jovyan/work/code')
from processing_helpers import *
from analysis_helpers import *
from gradientVersion import *
from maps_null_test import *
%R source("../code/Rplots.R");
%R source("../code/brainPlots.R");
%R source("../code/plot_maps.R");

Get components

In [54]:
dme_3d_ds5 = gradientVersion().fit('hcp_3d_ds5')

New gradients version: method=dm, kernel=normalized_angle, sparsity=0, data=hcp_3d_ds5


## Option one: make null scores from simulated maps

In [ ]:
dme_3d_ds5.make_null_scores(n=5000, save_name='hcp_3d_ds5_clean_sim_5000')

## Option two: make null scores from spins

First generate generic spins at given density and save

In [63]:
generate_spins(n=5000, blocks=1, density='41k', save_name = 'spins_41k_5000_v2')

                            00
Saved spins to ../outputs/permutations/spins_41k_5000_v2.npy

Generated 1 blocks of 5000 spins at density 41k


In [94]:
spins = np.load("../outputs/permutations/spins_41k_5000.npy")
# spins = np.load("../outputs/permutations/spins_41k_100.npy")

Read HCP parcellation files

In [5]:
hcp_img_files = ('../data/parcellations/lh.HCPMMP1.annot',
                 '../data/parcellations/rh.HCPMMP1.annot')
hcp_img = annot_to_gifti(hcp_img_files)

Project HCP to fsaverage at given density (e.g. 41k)

In [26]:
# hcp_img_10k = fsaverage_to_fsaverage(hcp_img, target_density='10k', method='nearest')
hcp_img_41k = fsaverage_to_fsaverage(hcp_img, target_density='41k', method='nearest')

Downloaded 5898240 of 6550379 bytes (90.0%,    0.1s remaining) ...done. (2 seconds, 0 min)
Extracting data from /home/jovyan/neuromaps-data/9e29c30a00cc86995f334ba48854d362/fsaverage41k.tar.gz..... done.


Project components to spins and reaverage

In [65]:
scores = dme_3d_ds5.scores
generate_nulls_from_components(scores, spins, parcellation_img = hcp_img_41k, density='41k',
                               n=5000, save_name = 'dme_3d_ds5_clean_spin_41k_5000')


Generated 5000 null spins of component 0

Generated 5000 null spins of component 1

Generated 5000 null spins of component 2
Saved null spins to ../outputs/permutations/dme_3d_ds5_clean_spin_41k_5000_v2.npy


Project DK components to spins and reaverage

In [78]:
dk_img = ("../data/parcellations/lh.aparc.annot", "../data/parcellations/rh.aparc.annot")
dk_img = annot_to_gifti(dk_img)
dk_img = fsaverage_to_fsaverage(dk_img, target_density='41k', method='nearest')

In [95]:
scores = dme_3d_ds5.score_in_dk()
generate_nulls_from_components(scores, spins, atlas='dk', parcellation_img = dk_img, density='41k',
                               n=5000, save_name = 'dme_3d_ds5_clean_in_dk_spin_41k_5000')


Generated 5000 null spins of component 0

Generated 5000 null spins of component 1

Generated 5000 null spins of component 2
Saved null spins to ../outputs/permutations/dme_3d_ds5_clean_in_dk_spin_41k_5000.npy


Project HCP spins to DK

In [66]:
nulls_hcp = np.load("../outputs/permutations/dme_3d_ds5_clean_spin_41k_5000_v2.npy")

In [67]:
dk_img_path = ('../data/parcellations/lh.aparc.annot',
               '../data/parcellations/rh.aparc.annot')
dk_img = annot_to_gifti(dk_img_path)
dk_img_41k = fsaverage_to_fsaverage(dk_img, target_density='41k', method='nearest')

In [70]:
from tqdm import tqdm
nulls_dk = np.zeros([34, 3, 5000])
for i in tqdm(range(nulls_hcp.shape[2])):
    scores_ = nulls_hcp[:,:,i]
    nulls_dk[:,:,i] = hcp_to_dk(scores_, 
                                hcp_img = (hcp_img_41k[0],), 
                                dk_img = (dk_img_41k[0],),
                                as_df=False)

100%|██████████| 5000/5000 [22:05<00:00,  3.77it/s]


In [71]:
np.save("../outputs/permutations/dme_3d_ds5_clean_spin_41k_5000_v2_dk.npy", nulls_dk)